---

### for me

 - [YAML Cheatsheet](https://quickref.me/yaml.html)
 - [YAML Viewer](https://jsonformatter.org/yaml-viewer)

# Notes

 - provide links and URIs wherever possible
 - _extended_ YAML with inline links in Markdown style [text](https://www.example.org)
 - 

# Feedback

 - the more you annotate _what_ the information in the guides _is_ (types, meta-info), the more information we can extract and potentially link together with the Datahub
 - try to keep structures _across_ guides of different levels _and_ pieces of information as uniform as possible
   -> makes info _predictable_ for others and allows to link more into the Datahub
 - 

# Problems

 - which URIs?
   -> definitely the Datahub ones _at some point_ but those aren't ready yet
  - which thesaurus/thesauri?

# Thoughts, Ideas

 - link directly to other search guides (across levels)
 - link directly to example objects that the current guide pertains to (is examplary of)
   -> this serves the "points of acces"/"portals" motivation of the research guides

---

In [4]:
import os

os.path.split("hello/world/whatever")

('hello/world', 'whatever')

# YAML to Markdown Parsing - custom-built

In [1]:
from glob import glob
import yaml

from ResearchAids import ResearchAid

BASE_DIR = "../published"
eng = glob(f"{BASE_DIR}/*/English/*.yml")
dutch = glob(f"{BASE_DIR}/*/Dutch/*.yml")
# top = glob(f"{BASE_DIR}/TopLevel/*.yml")

yaml_files = sorted(dutch + eng)

for filename in yaml_files:
    with open(filename) as handle:
        yml = yaml.safe_load(handle)
        try:
            r = ResearchAid(yml, raise_parsing_error=True)
        except KeyError as e:
            print(filename, e)
            # raise
            if "remarks" in str(e):
                print(filename, e)
                raise
            else:
                print(filename, e)
        except AttributeError as e:
            print(filename, e)



In [ ]:
from ResearchAids import *


Level2.parse_relevant_data("hello")

In [ ]:
for y in yamls:
    print(y.keys())

In [ ]:
# print(YAML2MD(yamls[1])())
# yamls[3]["Relevant data"]#["Tags"]
print([s.keys() for s in yamls[2]["Sources"]['Secondary sources']][0], "\n---")
print([s.keys() for s in yamls[3]["Sources"]['Secondary sources']][0])



In [ ]:
# print(Level2(yamls[2])())

print(ResearchAid(yamls[0], raise_parsing_error=True)())

In [ ]:
with open("../published/niveau3/English/NZG_20240508.yml") as handle:
    yml = yaml.safe_load(handle)

[d.keys() for d in yml["Sources"]['Secondary sources']]
yml["Sources"]['Secondary sources']

---

In [ ]:
import re
from glob import glob
import yaml


def correct_IRI(url):
    # correct IRIs:
    #  - https://sws.geonames.org/6255149/
    #  - http://vocab.getty.edu/aat/300266789
    # -- http://www.wikidata.org/entity/Q219477
    md_link_re = re.compile(r"\[(.*)\]\(https?:\/\/(?:sws|www).geonames.org\/([0-9]+)\/?.*\)")
    # uri_re = re.compile(r"^https?:\/\/(?:sws|www).geonames.org\/([0-9]+)\/?.*")

    if md_link_re.match(url):
        link_text, geonames_id = md_link_re.match(url).group(1), md_link_re.match(url).group(2)
        print(f"parsed {url}")
        return f"[{link_text}](https://sws.geonames.org/{geonames_id}/)"
    elif ("http" in url[:20]) or ("www" in url[:20]):
        print(f" {url}  didn't parse! is it correct?")
    else:
        pass
    
    return url



complex_types = (list, dict)
def iter_urls(yml):
    if isinstance(yml, str):
        return correct_IRI(yml)
    if isinstance(yml, list):
        return list(map(iter_urls, yml))
    if isinstance(yml, dict):
        return {iter_urls(k): iter_urls(v) for k, v in yml.items()}
    return yml

In [ ]:
BASE_DIR = "../published"
eng = glob(f"{BASE_DIR}/*/English/*.yml")
dutch = glob(f"{BASE_DIR}/*/Dutch/*.yml")
# top = glob(f"{BASE_DIR}/TopLevel/*.yml")

yaml_files = sorted(dutch + eng)

for filename in yaml_files:
    print(filename)
    with open(filename) as handle:
        yml = yaml.safe_load(handle)
        iter_urls(yml)
    print("\n-------------------\n")

In [ ]:
with open("../published/niveau0/Dutch/TopLevel_20240606.yml") as handle:
    yml = yaml.safe_load(handle)
    print(yml == iter_urls(yml))


---

# parsing MD to DOCX

In [ ]:
import yaml
from ResearchAids import ResearchAid

In [ ]:
aid = "../published/niveau3/English/WMLeiden_20240508.yml"
md_file = "../EXPORTS/MD/niveau3/English/WMLeiden.md"

with open(md_file) as handle:
    md = handle.read()

with open(aid) as handle:
    yml = yaml.safe_load(handle)


# print(md)

In [ ]:
ra = ResearchAid(yml)
if ra._parsed:
    md_content = ra()
    
    with open("test.md", "w") as handle:
        handle.write(md_content)

In [ ]:
import re

img_regex = re.compile(r"!\[.+\]\(.+\)")

md2 = md_content[:]
for instance in img_regex.findall(md2):
    md2 = md2.replace(instance, instance[1:])


In [ ]:
print(md2)

In [ ]:
from datetime import datetime



---
# related aids table

In [96]:
ymls = []
for filename in yaml_files:
    with open(filename) as handle:
        yml = yaml.safe_load(handle)
        ymls.append(yml)

curaid = ymls[10]
relaids = curaid["RelatedAides"]

In [97]:
def parse_related(ls):
    levels = [[] for _ in range(4)]
    for aid_d in ls:
        item_title, item_value_dict = tuple(aid_d.items())[0]
        link = item_value_dict["link"]
        rel_type = item_value_dict["rel_type"]
        
        lvl = int(link.split("/")[0][-1])
        levels[lvl].append((item_title, link))
    return levels

def empty_row(n_cols, fill="     "):
    return fill.join(["|"]*n_cols)

def md_row(cells, total_cols):
    pad = max((total_cols - len(cells))//2, 0) + 1

    row_md = empty_row(pad) or "|"
    for val in cells:
        row_md += f"  {val}  |"
    row_md += empty_row(pad + (total_cols - len(cells)) % 2)[1:]
    return row_md

def md_link(tup):
    text, link = tup
    if link is None:
        return f"_{text}_"
    return f"[{text}]({link})"

def related_aids_table(own_title, own_level, related_aids_ls):
    grouped_rel_aids = parse_related(related_aids_ls)
    grouped_rel_aids[own_level].insert(len(grouped_rel_aids[own_level])//2, (own_title, None))

    n_cols = max(map(len, grouped_rel_aids))

    md = empty_row(n_cols + 1) + "\n"
    md += empty_row(n_cols + 1, fill=":---:") + "\n"
    for i, lvl_ls in enumerate(grouped_rel_aids):
        # lvl_ls = sorted(lvl_d.items())

        # if len(lvl_ls) < 1: 
        #     md += empty_row(n_cols) + "\n"
        if True:
            links = list(map(md_link, sorted(lvl_ls)))
            md += md_row(links, n_cols) + "\n"
        

        # md += "|"
        # for tit, link in sorted(lvl_ls):
        #     md += f" [{tit}]({link}) |"
        # md += "\n"
    return md

In [98]:
print(related_aids_table(curaid["Title"], int(curaid["Level"]), relaids))

|     |     |     |     |     |
|:---:|:---:|:---:|:---:|:---:|
|     |     |     |     |     |
|     |  [Doing research](niveau1/English/DoingResearch_20240425.yml)  |  [Introduction to conducting provenance research](niveau1/English/Introduction_20250120.yml)  |  _Select and delineate_  |     |
|  [Army and navy personnel in colonised territories](niveau2/English/MilitaryAndNavy_20240417.yml)  |  [Christian missionaries in colonised territories](niveau2/English/ChristianMission_20240417.yml)  |  [Civil servants in colonised territories](niveau2/English/CivilServants_20240316.yml)  |  [Scientific research in colonised territories](niveau2/English/Science_20240821.yml)  |  [The trade in objects from a colonial context](niveau2/English/Trade_20240316.yml)  |
|     |     |     |     |     |



In [22]:
relaids

[{'Civil servants in colonised territories': {'link': 'niveau2/English/CivilServants_20240316.yml',
   'rel_type': 'see also'}},
 {'Scientific research in colonised territories': {'link': 'niveau2/English/Science_20240821.yml',
   'rel_type': 'see also'}},
 {'The trade in objects from a colonial context': {'link': 'niveau2/English/Trade_20240316.yml',
   'rel_type': 'see also'}},
 {'Artis Ethnographic Museum': {'link': 'niveau3/English/EMArtis_20240712.yml',
   'rel_type': 'see also'}},
 {'Wereldmuseum Amsterdam': {'link': 'niveau3/English/WMAmsterdam_20240809.yml',
   'rel_type': 'see also'}},
 {'Wereldmuseum Leiden': {'link': 'niveau3/English/WMLeiden_20240508.yml',
   'rel_type': 'see also'}},
 {'Wereldmuseum Rotterdam': {'link': 'niveau3/English/WMRotterdam_2040822.yml',
   'rel_type': 'see also'}},
 {'Rijksmuseum Amsterdam': {'link': 'niveau3/English/RijksmuseumAmsterdam_20240905.yml',
   'rel_type': 'see also'}},
 {'Museum Nusantara': {'link': 'niveau3/English/MNusantara_20250225

In [23]:
"" or "|"

'|'